**POZNÁMKA: Tento notebook je určený pre platformu Google Colab, ktorá zdarma poskytuje hardvérovú akceleráciu. Je však možné ho spustiť (možno s drobnými úpravami) aj ako štandardný Jupyter notebook, pomocou lokálnej grafickej karty.** 



In [ ]:
#@title -- Installation of Packages -- { display-mode: "form" }
import sys
import shutil
USE_NBCAP = False

if not shutil.which('apt') is None:
    !apt update
    !apt install -y xvfb x11-utils
    !apt install swig
    !{sys.executable} -m pip install pyscreenshot pyvirtualdisplay
    !{sys.executable} -m pip install --upgrade pyglet
    !{sys.executable} -m pip install git+https://github.com/michalgregor/nbcap.git

    USE_NBCAP = True

!{sys.executable} -m pip install gymnasium[box2d]
!{sys.executable} -m pip install class_utils[tensorboard]@git+https://github.com/michalgregor/class_utils.git
!{sys.executable} -m pip install pip install "ray[rllib]"

In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
import shutil
if shutil.which('apt') is None:
    USE_NBCAP = False
else:
    USE_NBCAP = True

    from nbcap import ShowVideoCallback, ScreenRecorder, OutputManager, DisplayProcess
    from pyscreenshot import grab as img_grab  

import gymnasium as gym
from ray.tune.registry import register_env

_lunar_max_episode_steps = 250

def timed_lunar_lander(**kwargs):
    env = gym.make('LunarLander-v2', **kwargs)
    env = gym.wrappers.TimeLimit(env,
        max_episode_steps=_lunar_max_episode_steps)
    return env

gym.envs.registration.register(
     id='TimedLunarLander-v0',
     entry_point='__main__:timed_lunar_lander',
 )

register_env("TimedLunarLander-v0", timed_lunar_lander)

In [ ]:
#@title -- Auxiliary Functions -- { display-mode: "form" }

if USE_NBCAP:
    display_size=(600, 400)
    show_video = ShowVideoCallback(dimensions=display_size)

    # make sure that only one instance
    # of the display is ever created
    try:
        DISP_PROC
    except NameError:
        DISP_PROC = DisplayProcess(display_size=display_size)

    def make_screen_recorder(max_gui_outputs=3):
        video_path="output"
        segment_time=10

        output_manager = OutputManager(max_gui_outputs=max_gui_outputs)
        video_callback=output_manager(show_video)
        display = DISP_PROC.id

        screen_recorder = ScreenRecorder(
            display, display_size, video_path,
            segment_time=segment_time, video_callback=video_callback
        )
        
        return screen_recorder

    SCREEN_RECORDER = make_screen_recorder()
else:
    from contextlib import suppress
    SCREEN_RECORDER = suppress()

    def img_grab():
        pass

## OpenAI Gym Prostredia

Predtým než preskúmame, ako funguje hlboké učenie s odmenou, poexperimentujeme najprv s jedným OpenAI Gym prostredím: s prostredím [Lunar Lander](https://gym.openai.com/envs/LunarLander-v2/) z [balíčka Gym od OpenAI](https://gym.openai.com/). Gym obsahuje viacero užitočných RL benchmarkových úloh, pričom všetky majú to isté unifikované rozhranie.

Inštanciu prostredia si vytvoríme volaním `gym.make('TimedLunarLander-v0')`. Špecifikujeme `render_mode='human'`, aby sa nám prostredie vizualizovalo.



In [ ]:
env = gym.make('TimedLunarLander-v0', render_mode='human')

Ak chceme začať novú epizódu a inicializovať jej počiatočný stav, zavoláme funkciu `reset()`. Táto navráti naše prvé pozorovanie stavu a info slovník (ktorý budeme v tomto prípade ignorovať).

Nebudeme sa teraz detailne venovať tomu, čo znamenajú jednotlivé hodnoty pozorovania. Ako príklad však povedzme, že prvé dve čísla zodpovedajú súradniciam lunárneho modulu, s ktorým sa budeme snažiť pristáť.



In [ ]:
obs, info = env.reset()
obs

O tvare a hraniciach priestoru pozorovaní sa môžeme viac dozvedieť prostredníctvom členskej premennej `env.observation_space`. V našom prípade zistíme, že pozorovania sa skladajú z 8 čísel.



In [ ]:
env.observation_space.shape

Hovorí sa, že lepšie raz vidieť než sto ráz počuť, preto si počiatočný stav aj vizualizujme. Samotná vizualizácia sa v skutočnosti už realizovala – lenže na strane vzdialeného servera, keďže kód spúšťame tam. Výsledok si môžeme zobraziť tak, že vytvoríme screenshot vzdialenej obrazovky pomocou funkcie `img_grab`.



In [ ]:
img_grab()

Ako ďalší krok sa pozrime, z akých akcií bude mať náš agent na výber. Zistiť to môžeme pomocou členskej premennej `env.action_space`. V našom prípade je priestor akcií diskrétny a skladá sa zo 4 rôznych akcií, ktorých význam je takýto:

* **0:**  nerobiť nič;
* **1:**  aktivovať pravý orientačný motor;
* **2:**  aktivovať hlavný motor;
* **3:**  aktivovať ľavý orientačný motor.


In [ ]:
env.action_space

Akcie sa vykonávajú volaním `env.step(action)`. Táto funkcia sa postará o vykonanie akcie a tiež navráti n-ticu obsahujúcu:

* `obs`: nasledujúce pozorovanie;
* `reward`: okamžitú odmenu;
* `terminated`: či sa prostredie prirodzene ukončilo;
* `truncated`: či sa prostredie ukončilo predčasne;
* `info`: slovník s ďalšími informáciami špecifickými pre daný typ prostredia.
Skúsme teraz niekoľko krokov nerobiť nič (akcia 0) a sledujme, čo sa stane s lunárnym modulom.



In [ ]:
obs, reward, terminated, truncated, info = env.step(0)
print("observation:", obs)
print("immediate reward:", reward)

Aby bol rozdiel markantnejší, spustíme tú istú akciu viackrát kým si budeme stav znovu vizualizovať. Malo by byť zrejmé, že lunárny modul zostúpil nižšie.



In [ ]:
for i in range(60):
    env.step(0)

img_grab()

Ak chcete, môžete teraz skúsiť s rozhraním ďalej experimentovať, možno sa dokonca pokúsiť ručne navrhnúť regulátor.

